In [19]:
#tensorflow_version 2.x 
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf     
import pandas as pd


CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
#Constants defined to help us

train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
#We use "keras"(a module inside tensorflow) to grab our datasets and read them into a pandas dataframe

print(train.head())
train_y = train.pop('Species')
test_y = test.pop('Species')
print(train.head())

print(train.shape)

   SepalLength  SepalWidth  PetalLength  PetalWidth  Species
0          6.4         2.8          5.6         2.2        2
1          5.0         2.3          3.3         1.0        1
2          4.9         2.5          4.5         1.7        2
3          4.9         3.1          1.5         0.1        0
4          5.7         3.8          1.7         0.3        0
   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3
(120, 4)


In [20]:
#Input Function
def input_fn(features, labels, training=True, batch_size=256):
    #Convert inputs to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    #shuffle and repeat if you are in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

In [21]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [22]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(4,)),  # Input layer
    tf.keras.layers.Dense(10, activation='relu'),  # Hidden layer 1
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train, train_y, epochs=5, batch_size=32)

Epoch 1/5


c:\Users\royav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6027 - loss: 1.3651  
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4596 - loss: 1.1352 
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3910 - loss: 0.9913
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7092 - loss: 0.9088 
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.7279 - loss: 0.8490


In [23]:
# Evaluate the model
#eval_result = model.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
eval_result = model.evaluate(test, test_y)
print('\nTest Set Accuracy: {accuracy:0.3f}\n'.format(accuracy=eval_result[1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.5333 - loss: 0.9581

Test Set Accuracy: 0.533



In [24]:
# Prediction
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict={}

print("Please type numeric values as prompted.")
for feature in features:
    valid = False
    while not valid:
        val = input(feature+": ")
        if val.replace('.', '', 1).isdigit():
            valid = True
        else:
            print("Please enter a valid numeric value.")
        
    predict[feature] = [float(val)]

#predictions = model.predict(input_fn=lambda: input_fn(predict, labels=None, training=False))
predictions = model.predict(pd.DataFrame(predict))
class_id = tf.argmax(predictions, axis=1).numpy()[0]
probability = predictions[0][class_id]

print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100*probability))

Please type numeric values as prompted.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Prediction is "Virginica" (45.9%)
